# **Tugas 7 - Diabetic Retinophaty**

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%cd /content/drive/MyDrive/KULIAH/SEMESTER 5/DATA MINING/datamining

In [ ]:
!pip install -U scikit-learn

In [ ]:
from scipy.io import arff
import pandas as pd

data = arff.loadarff('messidor_features.arff')
df = pd.DataFrame(data[0])

df.head()

In [ ]:
#read the data from csv file
col_names = []
for i in range(20):
    if i == 0:
        col_names.append('quality')
    if i == 1:
        col_names.append('prescreen')
    if i >= 2 and i <= 7:
        col_names.append('ma' + str(i))
    if i >= 8 and i <= 15:
        col_names.append('exudate' + str(i))
    if i == 16:
        col_names.append('euDist')
    if i == 17:
        col_names.append('diameter')
    if i == 18:
        col_names.append('amfm_class')
    if i == 19:
        col_names.append('label')
    
data = arff.loadarff('messidor_features.arff')
df = pd.DataFrame(data[0])

df.head()

In [ ]:
df.columns = [col_names]
df

In [ ]:
X = df.drop(columns=['label'])
X

## **Normalisasi**
Rumus normalisasi min-max  
$$ x' = \frac{x-x_{min}}{x_{max}-x_{min}}$$


In [ ]:
#normalisasi
from sklearn.preprocessing import MinMaxScaler
import joblib

scaler = MinMaxScaler()
# scaler.fit(features)
# scaler.transform(features)
scaled = scaler.fit_transform(X)
features_names = X.columns.copy()
# features_names.remove('label')
scaled_features = pd.DataFrame(scaled, columns=features_names)


#save model preprocessing
scaler_filename = "scaled.save"
joblib.dump(scaler, scaler_filename) 
scaler = joblib.load(scaler_filename)

scaled_features.head(10)
datates = arff.loadarff('messidor_features.arff')

In [ ]:
scaled_features

In [ ]:
scaled

In [ ]:
y = df['label'].values 
y[0:5]

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder() #untuk merubah label tipe kategori ke numerik scara berurutan
le.fit(y)

In [ ]:
ybaru = le.transform(y)
ybaru

## **Menggunakan Naive-Bayes Gaussian**

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer, accuracy_score,precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.preprocessing import LabelEncoder

#Model Select
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

#split dataset
training, test = train_test_split(scaled, train_size = 0.8, test_size = 0.2, shuffle=False)
training_label, test_label = train_test_split(ybaru, train_size = 0.8, test_size = 0.2, shuffle=False) 

clf2 = GaussianNB()
clf2.fit(training, training_label)

probas = clf2.predict_proba(test)
probas = probas[:,1]
probas.shape
probas

 

In [ ]:
probas = probas.round()
probas

In [ ]:
accuracy_score(test_label, probas)

In [ ]:
filemodelGaussian = 'modelGaussian.pkl'
joblib.dump(clf2, filemodelGaussian) #save mode gaussian

## **menggunakan metode K-NN**

In [ ]:
# splitting the data into training and test sets (80:20)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,ybaru,test_size=0.2,random_state=4)

In [ ]:
#shape of train and test objects
print(X_train.shape)
print(X_test.shape)

In [ ]:
#import the KNeighborsClassifier class from sklearn
from sklearn.neighbors import KNeighborsClassifier

#import metrics model to check the accuracy 
from sklearn import metrics
#Try running from k=1 through 25 and record testing accuracy
k_range = range(1,26)
scores = {}
scores_list = []
for k in k_range:
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train,y_train)
        y_pred=knn.predict(X_test)
        scores[k] = metrics.accuracy_score(y_test,y_pred)
        scores_list.append(metrics.accuracy_score(y_test,y_pred))

In [ ]:
scores

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

#plot the relationship between K and the testing accuracy
plt.plot(k_range,scores_list)
plt.xlabel('Value of K for KNN')
plt.ylabel('Testing Accuracy')

### **Bagging clasifier**


In [ ]:
gaussian = clf2.fit(training, training_label)

In [ ]:
training_pred = clf2.predict(training)
testing_pred = clf2.predict(test)



In [ ]:
clf_training = accuracy_score(training_label, training_pred)
clf_test = accuracy_score(test_label, testing_pred)

print("Gaussian Training",clf_training)
print("Gaussian Testing",clf_test)

In [ ]:
from sklearn.ensemble import BaggingClassifier
n = list(range(2,500))
acuracy = []
for i in n:
  bag = BaggingClassifier(
          base_estimator=gaussian, # knn,gnb, jst
          n_estimators=i,
          random_state=0)
  bag = bag.fit(training, training_label)
  y_test_pred = bag.predict(test)
  bag_test = accuracy_score(test_label, y_test_pred)
  print(f'acuracy n = {i} = {bag_test}')
  acuracy.append(bag_test)

acuracy n = 2 = 0.6363636363636364
acuracy n = 3 = 0.6363636363636364
acuracy n = 4 = 0.6363636363636364
acuracy n = 5 = 0.6233766233766234
acuracy n = 6 = 0.6493506493506493
acuracy n = 7 = 0.6406926406926406
acuracy n = 8 = 0.6406926406926406
acuracy n = 9 = 0.6406926406926406
acuracy n = 10 = 0.6320346320346321
acuracy n = 11 = 0.6406926406926406
acuracy n = 12 = 0.6363636363636364
acuracy n = 13 = 0.6406926406926406
acuracy n = 14 = 0.6320346320346321
acuracy n = 15 = 0.6320346320346321
acuracy n = 16 = 0.6363636363636364
acuracy n = 17 = 0.6277056277056277
acuracy n = 18 = 0.6277056277056277
acuracy n = 19 = 0.6277056277056277
acuracy n = 20 = 0.6277056277056277
acuracy n = 21 = 0.6277056277056277
acuracy n = 22 = 0.6363636363636364
acuracy n = 23 = 0.6363636363636364
acuracy n = 24 = 0.6363636363636364
acuracy n = 25 = 0.6406926406926406
acuracy n = 26 = 0.6406926406926406
acuracy n = 27 = 0.6406926406926406
acuracy n = 28 = 0.6406926406926406
acuracy n = 29 = 0.6406926406926406


In [ ]:
import matplotlib.pyplot as plt
# n = list(range(2,500))
plt.plot(n, acuracy)
plt.show()

In [ ]:
max(acuracy)

# **Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(n_estimators=14, max_depth=2, random_state=0)

In [ ]:
# bag = bag.fit(X_train, y_train)
clf = clf.fit(training, training_label)

In [ ]:
n = list(range(2,500))
acuracy = []
for i in n:
  clf = RandomForestClassifier(n_estimators=i, max_depth=2, random_state=0)
  clf = clf.fit(training, training_label)
  y_test_pred = clf.predict(test)
  bag_test = accuracy_score(test_label, y_test_pred)
  print(f'acuracy n = {i} = {bag_test}')
  acuracy.append(bag_test)

In [ ]:
import matplotlib.pyplot as plt
# n = list(range(2,500))
plt.plot(n, acuracy)
plt.show()

In [ ]:
max(acuracy)
# stacking